<a href="https://colab.research.google.com/github/Byeon-MJ/Face_Mask_Detection/blob/main/Mask_Detector_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mask Detect Model

## Module Import

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from sklearn.model_selection import train_test_split
import torchsummary
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np
import cv2
import json
import os
import shutil
import glob
import time
import copy
import random
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Dataset Preparation

### Initial Dataset Download - git clone
https://github.com/prajnasb/observations

In [ ]:
# 초기 데이터 git clone
!git clone https://github.com/prajnasb/observations/

# 분류된 데이터셋 드라이브에 저장
!mkdir /content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset
!cp -r /content/observations/experiements/dest_folder/test /content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset
!cp -r /content/observations/experiements/dest_folder/train /content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset

Cloning into 'observations'...
remote: Enumerating objects: 1638, done.
remote: Total 1638 (delta 0), reused 0 (delta 0), pack-reused 1638
Receiving objects: 100% (1638/1638), 75.94 MiB | 40.02 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [ ]:
# 초기 데이터셋 크기 확인
train_path_with_mask = glob.glob('/content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset/train/with_mask/*')
train_path_without_mask = glob.glob('/content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset/train/without_mask/*')
test_path_with_mask = glob.glob('/content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset/test/with_mask/*')
test_path_without_mask = glob.glob('/content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset/test/without_mask/*')

print(f'Train with_mask : {len(train_path_with_mask)}')
print(f'Train without_mask : {len(train_path_without_mask)}')
print(f'Test with_mask : {len(test_path_with_mask)}')
print(f'Test without_mask : {len(test_path_without_mask)}')

Train with_mask : 658
Train without_mask : 657
Test with_mask : 97
Test without_mask : 97


### Additional Dataset Download - Kaggle API
https://www.kaggle.com/datasets/omkargurav/face-mask-dataset

In [ ]:
# kaggle API Token 업로드
from google.colab import files
files.upload()

# kaggle API 준비
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Copy API Command
!kaggle datasets download -d omkargurav/face-mask-dataset

Saving kaggle.json to kaggle.json
 94% 153M/163M [00:01<00:00, 158MB/s]
100% 163M/163M [00:01<00:00, 160MB/s]


In [ ]:
# 파일 압축풀기
!unzip /content/face-mask-dataset.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: data/with_mask/with_mask_3297.jpg  
  inflating: data/with_mask/with_mask_3298.jpg  
  inflating: data/with_mask/with_mask_3299.jpg  
  inflating: data/with_mask/with_mask_33.jpg  
  inflating: data/with_mask/with_mask_330.jpg  
  inflating: data/with_mask/with_mask_3300.jpg  
  inflating: data/with_mask/with_mask_3301.jpg  
  inflating: data/with_mask/with_mask_3302.jpg  
  inflating: data/with_mask/with_mask_3303.jpg  
  inflating: data/with_mask/with_mask_3304.jpg  
  inflating: data/with_mask/with_mask_3305.jpg  
  inflating: data/with_mask/with_mask_3306.jpg  
  inflating: data/with_mask/with_mask_3307.jpg  
  inflating: data/with_mask/with_mask_3308.jpg  
  inflating: data/with_mask/with_mask_3309.jpg  
  inflating: data/with_mask/with_mask_331.jpg  
  inflating: data/with_mask/with_mask_3310.jpg  
  inflating: data/with_mask/with_mask_3311.jpg  
  inflating: data/with_mask/with_mask_3312.jpg  
  inflating: data/with_mask/with_mask

In [ ]:
# 추가 데이터셋을 Train, Test로 나누고 기존 경로로 복사해서 합치는 메서드
def split(img_list, test_count, train_path, test_path):
    # 추가 Test 데이터셋 Sampling
    test_files = []
    for i in random.sample(img_list, test_count):
        test_files.append(i)
    
    # Test 데이터셋을 제외한 나머지를 Train 데이터셋으로 분류
    train_files = [x for x in img_list if x not in test_files]

    # 추가 Train 데이터셋을 기존 데이터셋의 경로로 복사
    for k in train_files:
        shutil.copy(k, train_path)

    # 추가 Test 데이터셋을 기존 데이터셋의 경로로 복사
    for c in test_files:
        shutil.copy(c, test_path)

In [ ]:
# 추가 데이터셋 파일 지정
with_mask = glob.glob('/content/data/with_mask/*')
without_mask = glob.glob('/content/data/without_mask/*')

# Train Dataset 경로
with_mask_train_path = '/content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset/train/with_mask'
without_mask_train_path = '/content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset/train/without_mask'

# Test Dataset 경로
with_mask_test_path = '/content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset/test/with_mask'
without_mask_test_path = '/content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset/test/without_mask'

# Test Dataset 비율 지정
with_mask_test_count = round(len(with_mask)*0.2)
without_mask_test_count = round(len(without_mask)*0.2)

In [ ]:
# 데이터셋 나누기, 기존 경로로 복사
split(with_mask, with_mask_test_count, with_mask_train_path, with_mask_test_path)
split(without_mask, without_mask_test_count, without_mask_train_path, without_mask_test_path)

In [ ]:
# 새로운 데이터셋 크기 확인
train_path_with_mask = glob.glob('/content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset/train/with_mask/*')
train_path_without_mask = glob.glob('/content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset/train/without_mask/*')
test_path_with_mask = glob.glob('/content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset/test/with_mask/*')
test_path_without_mask = glob.glob('/content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset/test/without_mask/*')

print(f'Train with_mask : {len(train_path_with_mask)}')
print(f'Train without_mask : {len(train_path_without_mask)}')
print(f'Test with_mask : {len(test_path_with_mask)}')
print(f'Test without_mask : {len(test_path_without_mask)}')

Train with_mask : 3638
Train without_mask : 3719
Test with_mask : 842
Test without_mask : 863


## DataLoader

In [ ]:
# 데이터셋 경로 지정
experiments_path = '/content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset'

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(), 
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]), 
    'test' : transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [ ]:
# ImageFolder : 폴더명을 Label로 하여 데이터셋 생성
image_datasets = {x: datasets.ImageFolder(os.path.join(experiments_path, x), data_transforms[x]) for x in ['train', 'test']}
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 7357
     Root location: /content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset/train
     StandardTransform
 Transform: Compose(
                RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear), antialias=None)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ), 'test': Dataset ImageFolder
     Number of datapoints: 1705
     Root location: /content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset/test
     StandardTransform
 Transform: Compose(
                Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
                CenterCrop(size=(224, 224))
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            )}

In [ ]:
# ImageFolder으로 만든 데이터셋을 데이터로더로 전달
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=4) for x in ['train', 'test']}
dataloaders

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


{'train': <torch.utils.data.dataloader.DataLoader at 0x7efe42d4b8e0>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7efed8ba0be0>}

In [ ]:
class_names = image_datasets['train'].classes
class_names

['with_mask', 'without_mask']

In [ ]:
# device 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
dataset_sizes

{'train': 7357, 'test': 1705}

## Model Training

In [ ]:
# model train 메서드 생성
def train_model(model, criterion, optimizer, scheduler, num_epochs=20):
    since = time.time()  # 모델 훈련 시간 측정 위한 Time 모듈
    best_acc = 0.0
    best_model = copy.deepcopy(model.state_dict())
    

    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs - 1))
        print('-' * 10)
        
        # 매 epoch마다 훈련과 평가를 번갈아가며 시행
        for phase in ['train', 'test']:
            # 진행 상황 확인 위한 tqdm 사용
            iterator = tqdm(dataloaders[phase])
            
            # phase에 따른 모델 세팅 변경
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
                
            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in iterator:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()
                
                
                with torch.set_grad_enabled(phase == 'train'):
                    # forward
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)

                    # Loss 평가
                    loss = criterion(outputs, labels)
                    
                    # backward, optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print('{} Loss: {:.4f} Acc:{:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model = copy.deepcopy(model.state_dict())
            
            print()
    
    # 시작 시각과 종료 시각 측정하여 경과 시간 계산
    time_elapsed = time.time() - since
    print('Training complete in {:0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
    print('Best val acc: {:4f}'.format(best_acc))
    
    # 모델 학습중 가장 좋은 모델 불러오기
    model.load_state_dict(best_model)
    return model

In [ ]:
# 일부 이미지에 대한 예측값을 보여주는 메서드
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    #fig = plt.figure(figsize=(10,10))
    
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['test']):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            print(preds,"predicitons")
            
            
            for j in range(inputs.size()[0]):
                images_so_far +=1
                #ax = plt.subplot(num_images//len(labels)-1, len(labels), images_so_far)
                #ax.axis('off')
                #ax.set_title('true: {} predicted: {}'.format(class_names[labels[j]], class_names[preds[j]]))
                print('true: {} predicted: {}'.format(class_names[labels[j]], class_names[preds[j]]))
                #imshow(inputs.cpu().data[j])
                
                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

### ResNet101

In [ ]:
# 특정 url로 request할 때 ssl 인증 위한 임시 context 생성
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
# ResNet Model
resnet_model = models.resnet101(pretrained=True)

# FineTuning
# 미리 학습한 모델을 불러온 후 마지막의 완전히 연결된 계층을 재설정(reset)
num_frts = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(num_frts, len(class_names))

# cuda로 보내기
resnet_model = resnet_model.to(device)

# Loss Function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer_ft = optim.Adam(resnet_model.parameters(), lr=0.001)

# Scheduler
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

In [ ]:
import torchsummary
# ResNet모델 구조, 파라미터 수 확인
torchsummary.summary(resnet_model, (3, 224, 224), batch_size=16)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [16, 64, 112, 112]           9,408
       BatchNorm2d-2         [16, 64, 112, 112]             128
              ReLU-3         [16, 64, 112, 112]               0
         MaxPool2d-4           [16, 64, 56, 56]               0
            Conv2d-5           [16, 64, 56, 56]           4,096
       BatchNorm2d-6           [16, 64, 56, 56]             128
              ReLU-7           [16, 64, 56, 56]               0
            Conv2d-8           [16, 64, 56, 56]          36,864
       BatchNorm2d-9           [16, 64, 56, 56]             128
             ReLU-10           [16, 64, 56, 56]               0
           Conv2d-11          [16, 256, 56, 56]          16,384
      BatchNorm2d-12          [16, 256, 56, 56]             512
           Conv2d-13          [16, 256, 56, 56]          16,384
      BatchNorm2d-14          [16, 256,

In [ ]:
# 모델 학습
resnet_model = train_model(resnet_model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=20)

Epoch 0/19
----------


  0%|          | 0/460 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


train Loss: 0.3532 Acc:0.8506



  0%|          | 0/107 [00:00<?, ?it/s]

...

  0%|          | 0/460 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


train Loss: 0.0956 Acc:0.9575



  0%|          | 0/107 [00:00<?, ?it/s]

test Loss: 0.0330 Acc:0.9906

Training complete in 48.000000m 6s
Best val acc: 0.991202


In [ ]:
visualize_model(resnet_model.to(device))

tensor([1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0], device='cuda:0') predicitons
true: without_mask predicted: without_mask
true: with_mask predicted: with_mask
true: without_mask predicted: without_mask
true: without_mask predicted: without_mask
true: with_mask predicted: with_mask
true: with_mask predicted: with_mask


### MobileNetV2
- 모바일 환경 등에 사용하기 위해 경량화에 집중한 모델
- MobileNetV2: Inverted Residuals and Linear Bottlenecks(2018) 발표 논문 : 
    https://arxiv.org/abs/1801.04381

In [ ]:
mobilenet_model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True).to(device)

# MobileNet 전이학습 FineTuning - ResNet과 다름 주의
# 미리 학습한 모델을 불러온 후 마지막의 완전히 연결된 계층을 재설정(reset)
num_frts = mobilenet_model.classifier[1].in_features
mobilenet_model.classifier[1] = nn.Linear(num_frts, len(class_names))

mobilenet_model = mobilenet_model.to(device)
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(mobilenet_model.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [ ]:
# MobileNetV2모델 구조, 파라미터 수 확인
torchsummary.summary(mobilenet_model, (3, 224, 224), batch_size=16)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [16, 32, 112, 112]             864
       BatchNorm2d-2         [16, 32, 112, 112]              64
             ReLU6-3         [16, 32, 112, 112]               0
            Conv2d-4         [16, 32, 112, 112]             288
       BatchNorm2d-5         [16, 32, 112, 112]              64
             ReLU6-6         [16, 32, 112, 112]               0
            Conv2d-7         [16, 16, 112, 112]             512
       BatchNorm2d-8         [16, 16, 112, 112]              32
  InvertedResidual-9         [16, 16, 112, 112]               0
           Conv2d-10         [16, 96, 112, 112]           1,536
      BatchNorm2d-11         [16, 96, 112, 112]             192
            ReLU6-12         [16, 96, 112, 112]               0
           Conv2d-13           [16, 96, 56, 56]             864
      BatchNorm2d-14           [16, 96,

In [ ]:
# 모델 학습
mobilenet_model = train_model(mobilenet_model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=20)

Epoch 0/19
----------


  0%|          | 0/460 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


train Loss: 0.2518 Acc:0.8983



  0%|          | 0/107 [00:00<?, ?it/s]

...

  0%|          | 0/460 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


train Loss: 0.0761 Acc:0.9652



  0%|          | 0/107 [00:00<?, ?it/s]

test Loss: 0.0242 Acc:0.9947

Training complete in 19.000000m 18s
Best val acc: 0.995894


In [ ]:
visualize_model(mobilenet_model.to(device))

tensor([1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1], device='cuda:0') predicitons
true: without_mask predicted: without_mask
true: without_mask predicted: without_mask
true: with_mask predicted: with_mask
true: with_mask predicted: with_mask
true: without_mask predicted: without_mask
true: without_mask predicted: without_mask


## Model Save

In [ ]:
# 모델 저장
torch.save(resnet_model.to('cpu'), 'mask_detector_resnet101.pth')
torch.save(mobilenet_model.to('cpu'), 'mask_detector_mobilenet.pth')

In [ ]:
# 모델 Drive에 저장
!cp /content/mask_detector_resnet101.pth /content/gdrive/MyDrive/Project/Mask_Detection_Model/
!cp /content/mask_detector_mobilenet.pth /content/gdrive/MyDrive/Project/Mask_Detection_Model/

## Optimizer - AdamW
- CV Task에서는 Adam이 일반화가 많이 뒤쳐진다는 결과들
- AdamW가 Loss에서 더 좋은 성능을 보여준다는 논문 결과를 바탕으로 Optimizer를 변경하여 다시 시도 > 성능 향상

In [ ]:
mobilenet_model2 = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True).to(device)

# MobileNet 전이학습 FineTuning - ResNet과 다름 주의
# 미리 학습한 모델을 불러온 후 마지막의 완전히 연결된 계층을 재설정(reset)
num_frts = mobilenet_model2.classifier[1].in_features
mobilenet_model2.classifier[1] = nn.Linear(num_frts, len(class_names))

mobilenet_model2 = mobilenet_model2.to(device)
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.AdamW(mobilenet_model2.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [ ]:
# 모델 학습
mobilenet_model2 = train_model(mobilenet_model2, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=20)

Epoch 0/19
----------


  0%|          | 0/460 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


train Loss: 0.2324 Acc:0.9025



  0%|          | 0/107 [00:00<?, ?it/s]

...

  0%|          | 0/460 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


train Loss: 0.0746 Acc:0.9641



  0%|          | 0/107 [00:00<?, ?it/s]

test Loss: 0.0182 Acc:0.9971

Training complete in 19.000000m 24s
Best val acc: 0.997067
